In [1]:
import spacy
nlp = spacy.load("en_core_web_md")

In [2]:
import pandas as pd
test = pd.read_csv('testdata/Corona_NLP_test.csv')
tweets = test.OriginalTweet
tweets

0       TRENDING: New Yorkers encounter empty supermar...
1       When I couldn't find hand sanitizer at Fred Me...
2       Find out how you can protect yourself and love...
3       #Panic buying hits #NewYork City as anxious sh...
4       #toiletpaper #dunnypaper #coronavirus #coronav...
                              ...                        
3793    Meanwhile In A Supermarket in Israel -- People...
3794    Did you panic buy a lot of non-perishable item...
3795    Asst Prof of Economics @cconces was on @NBCPhi...
3796    Gov need to do somethings instead of biar je r...
3797    I and @ForestandPaper members are committed to...
Name: OriginalTweet, Length: 3798, dtype: object

'''
"toilet paper"
"grocery"
'long time'
"sanitizer"
''' 

In [3]:
tweets.describe()

count                                                  3798
unique                                                 3798
top       TRENDING: New Yorkers encounter empty supermar...
freq                                                      1
Name: OriginalTweet, dtype: object

In [4]:
import numpy as np

In [5]:
nptweets= tweets.to_numpy()

In [6]:
tweet1 = tweets[1]

In [7]:
doc = nlp(tweet1)

In [8]:
doc

When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY

In [9]:
clean_tweets = tweets.to_frame()

In [10]:
clean_tweets.to_csv("./clean_tweets.tsv", sep='\t', header=None, index=False)

In [11]:
def get_stopwords():
    "Return a set of stopwords read in from a file."
    with open("stopwords.txt") as f:
        stopwords = []
        for line in f:
            stopwords.append(line.strip("\n"))
    # Convert to set for performance
    stopwords_set = set(stopwords)
    return stopwords_set

stopwords = get_stopwords()

def save_work(df, file):
    return df.to_csv(file, sep='\t', header=None, index=False)

def remove_hashtags():
    # Some kind of regex which finds sequences of characters with a # or an @ at their start
    # And then removes the #, @ but keeps the remainder
    pass

def cleaner(df):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 3-50 chars
    pattern = re.compile(r"[A-Za-z0-9\-]{3,50}")
    df['clean'] = df['content'].str.findall(pattern).str.join(' ')
    if limit > 0:
        return df.iloc[:limit, :].copy()
    else:
        return df

def lemmatize(text):
    """Perform lemmatization in the OriginalTweets
       Returns a list of lemmas
    """
    doc = nlp(text)
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stopwords]
    return lemma_list

In [12]:
%%time
lemmatize(tweets[1])

CPU times: total: 15.6 ms
Wall time: 13 ms


['hand',
 'sanitizer',
 'fred',
 'meyer',
 'turn',
 'amazon',
 'pack',
 'coronaviru',
 'concern',
 'drive',
 'price']

In [13]:
clean_tweets

,OriginalTweet
0,TRENDING: New Yorkers encounter empty supermar...
1,When I couldn't find hand sanitizer at Fred Me...
2,Find out how you can protect yourself and love...
3,#Panic buying hits #NewYork City as anxious sh...
4,#toiletpaper #dunnypaper #coronavirus #coronav...
...,...
3793,Meanwhile In A Supermarket in Israel -- People...
3794,Did you panic buy a lot of non-perishable item...
3795,Asst Prof of Economics @cconces was on @NBCPhi...
3796,Gov need to do somethings instead of biar je r...


In [14]:
def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stopwords] 
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe


In [15]:
%%time
clean_tweets['preproc_pipe'] = preprocess_pipe(clean_tweets['OriginalTweet'])
clean_tweets.head(3)


KeyboardInterrupt: 

In [16]:

clean_tweets.columns

Index(['OriginalTweet'], dtype='object')

In [17]:
clean_tweets['preproc_pipe'].loc[4]


KeyboardInterrupt



In [ ]:
'supermarket' in clean_tweets['preproc_pipe'].loc[0]

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

In [ ]:
# shortage_patterns = [
#     [{"LEMMA": "sell"}, {"POS": "ADP"}],
#     [{"LEMMA": "buy"}, {"POS": "ADP"}],
#     [{"LOWER": "panic"}, {"LEMMA": "buy"}],
#     [{"LEMMA": "empty"}, {"LEMMA": "shelf"}],
#     [{"LEMMA": "stockpile"}],
#     [{"LOWER": "stock"}, {"LOWER": "pile"}],
#     [{"LEMMA": "price"}],
#     [{"LEMMA": "stock"}, {"POS": "ADP"}],
#     [{"LEMMA": "supermarket"}],
#     [{"LEMMA": "hoard"}],
#     [{"LEMMA": "hunger"}],
#     [{"LEMMA": "inflate"}],
#     [{"LOWER": {"REGEX": "c.*n't"}}]
# ]
# matcher.add("Shortage", shortage_patterns)

In [ ]:
# def match_pipe(texts):
#     match_pipe = []
#     for doc in nlp.pipe(texts, batch_size=20):
#         match_pipe.append(matcher(doc))
#     return match_pipe

# clean_tweets['matched_shortage'] = match_pipe(clean_tweets['OriginalTweet'])
# clean_tweets.head(6)

In [ ]:
# clean_tweets.head(6)

In [ ]:
# twt = clean_tweets[0:5]
# list = []

# for value in clean_tweets['matched_shortage']:
#     list.append(not value == [])
# clean_tweets['shortage_mask'] = pd.DataFrame(list)

In [ ]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LEMMA')
terms = ["hand sanitizer", "mask", "N95", "respirators", "lysol"]
ppe_patterns = [nlp(text) for text in terms]
matcher.add("PPE", ppe_patterns)

terms = ["sold out", "buy", "panic buy", "empty shelves", "stockpile", "inflated price", "stock", "supermarket", "hoarding", "hungry", "price"]
shortage_patterns = [nlp(text) for text in terms]
matcher.add("shortage", shortage_patterns)

terms = ["sick leave", "health insurance", "ICU", "hospital", "protection", "testing", "medicaid"]
medical_patterns = [nlp(text) for text in terms]
matcher.add("medical", medical_patterns)

In [ ]:
def match_pipe(texts):
    match_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        match_pipe.append(matcher(doc))
    return match_pipe

clean_tweets['matches'] = match_pipe(clean_tweets['OriginalTweet'])
clean_tweets.head(6)

In [ ]:
doc = nlp(clean_tweets['OriginalTweet'].loc[5])
doc[34:35]

In [ ]:
from spacy.strings import StringStore
stringstore = StringStore(["PPE", "medical", "shortage"])

In [ ]:
clean_tweets[["matches"]]

In [ ]:
df = clean_tweets

In [ ]:
categories_series = []

for match_list in df[["matches"]].itertuples(name=None):
    if match_list[1] == []:
        categories = {}
    else:
        categories = []
        for match_obj in match_list[1]:
            categories.append(stringstore[match_obj[0]]) 
        categories = set(categories)
    categories_series.append(categories)
    
cat = pd.Series(categories_series, name='categories')


In [ ]:
df = clean_tweets.join(cat)
df

In [ ]:
df = df[['OriginalTweet', 'categories']]

In [ ]:
# # Given df.categories, create masks 

# def mask(set):
#     "Takes a set and returns a dict of booleans for it"
#     dict = {'PPE': False, 'shortage': False, 'medical': false}
#     if 'PPE' in set:
#         dict['PPE'] = True
#     if 'medical' in set:
#         dict['medical'] = True
#     if 'shortage' in set: 
#         dict['shortage'] = True
#     return dict

In [ ]:
# list = []
# for set in df.categories:
#     list.append(len(set))
    
# information = pd.Series(list)
# information.value_counts()

In [ ]:
# df.categories

In [ ]:
# df[['PPE', 'shortage', 'medical']] = False
# df

In [ ]:
# # def better_mask(set, category): 
# #     if category not in set:
# #         return False
# #     else:
# #         return True

# df2 = df.head(3)
# print(df2)
    
# index = 0
# for set in df2.categories:
#     for x in set:
#         df2[x][index] = True
#     index += 1
    
#         # df2[[x]] = df2.categories.apply(better_mask, args = x)
# df2

In [ ]:
# index = 0
# for set in df.categories:
#     for x in set:
#         df[x][index] = True
#     index += 1

In [ ]:
important = df.categories.value_counts()

In [ ]:
important

In [ ]:
important.index = ['shortage', 'none', 'shortage, PPE', 'medical', 'PPE',
       'medical, shortage', 'medical, shortage, PPE']

In [ ]:
important.index